### Basic working of Google Palm LLM in LangChain

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install --upgrade --quiet  langchain-google-genai

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
import os

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env 


# get this free api key from https://makersuite.google.com/

#llm = GoogleGenerativeAI(model="models/gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

In [ ]:
llm

In [ ]:
llm.invoke("What do you think of rajma chawal?")

In [ ]:
essay = llm.invoke("write email requesting refund for electronic item")
print(essay)

In [ ]:
from langchain.chains import RetrievalQA

from langchain.embeddings import GooglePalmEmbeddings

### Now let's load data from xyz FAQ csv file

In [ ]:
# https://python.langchain.com/docs/integrations/document_loaders/csv/

from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='xyz_faqs.csv', source_column="prompt",encoding="latin-1")

# Store the loaded data in the 'data' variable
data = loader.load()

In [ ]:
data

### Hugging Face Embeddings

In [ ]:
# https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub/

%pip install --upgrade --quiet  langchain sentence_transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings()

query_result = hf_embeddings.embed_query('my name is Raman Khanna')

In [ ]:
len(query_result)

In [ ]:
query_result

### Vector store using FAISS

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=hf_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [ ]:
rdocs = retriever.invoke("how about job placement support?")
rdocs

In [ ]:
rdocs = retriever.invoke("do u guys have any relevant courses?")
rdocs

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case

In [ ]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀

In [ ]:
# from langchain.prompts import PromptTemplate

# prompt_template = """Given the following context and a question, generate an answer based on this context only.
# In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
# If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

# CONTEXT: {context}

# QUESTION: {question}"""


# PROMPT = PromptTemplate(
#     template=prompt_template, input_variables=["context", "question"]
# )
# chain_type_kwargs = {"prompt": PROMPT}


# from langchain.chains import RetrievalQA

# chain = RetrievalQA.from_chain_type(llm=llm,
#                             chain_type="stuff",
#                             retriever=retriever,
#                             input_key="query",
#                             return_source_documents=True,
#                             chain_type_kwargs=chain_type_kwargs)


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)



from langchain.chains import RetrievalQA
chain_type_kwargs = {"prompt": PROMPT}

#chain_type_kwargs: This is a dictionary that holds settings for the chain you will create. 

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

#return_source_documents: If set to True, the chain will return the documents it used to generate the answer.


# RetrievalQA, the chain_type parameter specifies the internal method used to handle the documents once they are retrieved. There are a few different strategies (or chain types) you might use:
# stuff: This approach concatenates all the retrieved documents into a single context and then passes this context to the language model to generate an answer.
# map-reduce: This approach first generates partial answers for each retrieved document and then combines these partial answers into a final answer.
# refine: This approach iteratively improves the answer by passing it through the language model multiple times, each time adding more information from the retrieved documents.


### We are all set 👍🏼 Let's ask some questions now

In [ ]:
chain.invoke('Do you provide job assistance and also do you provide job gurantee?')

In [ ]:
chain.invoke('Do you provide any javascript course?')

In [ ]:
chain('Do you provide emi options?')

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [ ]:
chain("Do you have plans to launch blockchain course in future?")

In [ ]:
chain("should I learn power bi or tableau?")

In [ ]:
chain("who is apj abdul kalam ?")

In [ ]:
chain.invoke("I've a MAC computer. Can I use powerbi on it?")